# Определение стоимости автомобилей

В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Необходимл построить модель для определения стоимости. 


## Подготовка данных

In [31]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from lightgbm import LGBMModel
import lightgbm
from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
data = pd.read_csv('/datasets/autos.csv')

In [3]:
display(data)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


In [5]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [6]:
data_1 = data.drop(['NumberOfPictures', 'PostalCode', 'DateCreated', 'LastSeen', 'DateCrawled'], axis = 1)

In [7]:
data_1 = data_1.fillna('Unknown')

In [8]:
data_1.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645
std,4514.158514,90.227958,189.850405,37905.341530,3.726421
min,0.000000,1000.000000,0.000000,5000.000000,0.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000


In [9]:
display(data_1)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired
0,480,Unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,Unknown
1,18300,coupe,2011,manual,190,Unknown,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,Unknown
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...,...
354364,0,Unknown,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes
354365,2200,Unknown,2005,Unknown,0,Unknown,20000,1,Unknown,sonstige_autos,Unknown
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


In [10]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 11 columns):
Price                354369 non-null int64
VehicleType          354369 non-null object
RegistrationYear     354369 non-null int64
Gearbox              354369 non-null object
Power                354369 non-null int64
Model                354369 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             354369 non-null object
Brand                354369 non-null object
NotRepaired          354369 non-null object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


In [11]:
data_2 = data_1.query("Price > 100 and RegistrationYear <= 2021 and Power <= 1000")

In [12]:
data_2.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
count,339673.000000,339673.000000,339673.000000,339673.000000,339673.000000
mean,4603.644697,2003.133658,108.233931,128528.216844,5.807097
std,4515.428210,11.721585,63.375156,37169.842998,3.685615
min,101.000000,1000.000000,0.000000,5000.000000,0.000000
25%,1200.000000,1999.000000,70.000000,125000.000000,3.000000
50%,2900.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6500.000000,2008.000000,143.000000,150000.000000,9.000000
max,20000.000000,2019.000000,1000.000000,150000.000000,12.000000


In [13]:
data_3 = pd.get_dummies(data_2, drop_first=True)

In [14]:
data_3['Price'].describe()

count    339673.000000
mean       4603.644697
std        4515.428210
min         101.000000
25%        1200.000000
50%        2900.000000
75%        6500.000000
max       20000.000000
Name: Price, dtype: float64

### Вывод
Представленные даныые были изучены.При анализе данных были удалены столбцы, не имеющие отношение к цене, т.к. их наличие только бы ухудшало качество модели. Пропущенные значения в категориальных признаках были заменены на значение Unknown. Был удалены строки с явно ошибочными значениями(год больше 2021, цена меньше 100 евро, мощность больше 1000 л.с.). Также была произведена кодировка признаков для работы с обычными моделями.

## Обучение моделей

In [15]:
features = data_3.drop('Price', axis = 1)
target = data_3['Price']
features_a, features_valid, target_a, target_valid = train_test_split(features, target, test_size=0.2, random_state=12345)
features_train, features_test, target_train, target_test = train_test_split(features_a, target_a, test_size=0.25, random_state=12345)

#### Линейная регрессия

In [16]:
model = LinearRegression(normalize = False)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions_valid)**0.5
print(rmse)

2865.7128232028253


#### Случайный лес

In [17]:
best_model = None
best_result = 2500
best_est = 0
for est in range(1, 11, 3):
    model = RandomForestRegressor(random_state=12345, n_estimators=est) 
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = mean_squared_error(target_valid, predictions_valid)**0.5
    if result < best_result:
        best_model = model
        best_result = result
        best_est = est
print('RMSE наилучшей модели на валидационной выборке:', best_result)
print('Значение параметра n_estimators для модели с лучшим RMSE:', best_est)

RMSE наилучшей модели на валидационной выборке: 1697.716759887157
Значение параметра n_estimators для модели с лучшим RMSE: 10


#### Решающее дерево

In [18]:
best_model = None
best_result = 2500
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeRegressor(random_state=12345, max_depth = depth)
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)
    result = mean_squared_error(target_valid, predictions_valid)**0.5
    if result < best_result:
        best_model = model
        best_result = result
        best_depth = depth
print('RMSE наилучшей модели на валидационной выборке:', best_result)
print('Значение параметра max_depth для модели с лучшей RMSE:', best_depth)

RMSE наилучшей модели на валидационной выборке: 2039.594454236832
Значение параметра max_depth для модели с лучшей RMSE: 10


#### Модель с градиентным бустингом с использованием библиотеки lightgbm

In [19]:
model = LGBMModel(objective="regression", n_estimators=500, learning_rate = 1)# Не нашел способ задать категориальные признаки
model.fit(features_train, target_train, eval_set=[(features_valid, target_valid)], eval_metric = 'rmse', verbose=100) 
predictions_valid = model.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions_valid)**0.5
print(rmse)

[100]	valid_0's rmse: 1712.61	valid_0's l2: 2.93303e+06
[200]	valid_0's rmse: 1712.42	valid_0's l2: 2.93237e+06
[300]	valid_0's rmse: 1716.86	valid_0's l2: 2.94761e+06
[400]	valid_0's rmse: 1726.27	valid_0's l2: 2.98e+06
[500]	valid_0's rmse: 1732.77	valid_0's l2: 3.00248e+06
1732.7659699459514


In [20]:
model = LGBMModel(objective="regression", n_estimators=500, learning_rate = 0.2)
model.fit(features_train, target_train, eval_set=[(features_valid, target_valid)], eval_metric = 'rmse', verbose=100) 
predictions_valid = model.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions_valid)**0.5
print(rmse)

[100]	valid_0's rmse: 1685.43	valid_0's l2: 2.84068e+06
[200]	valid_0's rmse: 1637.11	valid_0's l2: 2.68013e+06
[300]	valid_0's rmse: 1613.22	valid_0's l2: 2.60249e+06
[400]	valid_0's rmse: 1601.15	valid_0's l2: 2.56369e+06
[500]	valid_0's rmse: 1591.53	valid_0's l2: 2.53298e+06
1591.533082937891


In [21]:
model = LGBMModel(objective="regression", n_estimators=500, learning_rate = 0.4)
model.fit(features_train, target_train, eval_set=[(features_valid, target_valid)], eval_metric = 'rmse', verbose=100) 
predictions_valid = model.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions_valid)**0.5
print(rmse)

[100]	valid_0's rmse: 1655.33	valid_0's l2: 2.74011e+06
[200]	valid_0's rmse: 1621.5	valid_0's l2: 2.62928e+06
[300]	valid_0's rmse: 1603.46	valid_0's l2: 2.57108e+06
[400]	valid_0's rmse: 1593.71	valid_0's l2: 2.53991e+06
[500]	valid_0's rmse: 1590.71	valid_0's l2: 2.53037e+06
1590.7138564269912


#### Модель с градиентным бустингом с использованием библиотеки Catboost

In [22]:
features_x = data_2.drop('Price', axis = 1)
target_x = data_2['Price']
features_a, features_valid_x, target_a, target_valid_x = train_test_split(features_x, target_x, test_size=0.2, random_state=12345)
features_train_x, features_test_x, target_train_x, target_test_x = train_test_split(features_a, target_a, test_size=0.25, random_state=12345)
cat_features = ['VehicleType', 'RegistrationYear', 'Gearbox', 'Model', 'RegistrationMonth', 'FuelType', 'Brand', 'NotRepaired']               

In [23]:
model = CatBoostRegressor(loss_function="RMSE", iterations=200, learning_rate = 0.1) 
model.fit(features_train_x, target_train_x, cat_features=cat_features, verbose=20)
predictions_valid = model.predict(features_valid_x)
rmse = mean_squared_error(target_valid, predictions_valid)**0.5
print(rmse)

0:	learn: 4215.0747583	total: 777ms	remaining: 2m 34s
20:	learn: 2148.2399784	total: 16.7s	remaining: 2m 22s
40:	learn: 1948.5944636	total: 34.2s	remaining: 2m 12s
60:	learn: 1879.7159989	total: 51.2s	remaining: 1m 56s
80:	learn: 1837.1898450	total: 1m 8s	remaining: 1m 40s
100:	learn: 1806.2224230	total: 1m 25s	remaining: 1m 23s
120:	learn: 1784.4714156	total: 1m 42s	remaining: 1m 6s
140:	learn: 1766.9956712	total: 1m 59s	remaining: 50.1s
160:	learn: 1753.0430959	total: 2m 17s	remaining: 33.4s
180:	learn: 1740.4498249	total: 2m 35s	remaining: 16.3s
199:	learn: 1729.0551734	total: 2m 51s	remaining: 0us
1722.465875014314


In [24]:
model = CatBoostRegressor(loss_function="RMSE", iterations=200, learning_rate = 0.5) 
model.fit(features_train_x, target_train_x, cat_features=cat_features, verbose=20)
predictions_valid = model.predict(features_valid_x)
rmse = mean_squared_error(target_valid, predictions_valid)**0.5
print(rmse)

0:	learn: 3168.5984238	total: 728ms	remaining: 2m 24s
20:	learn: 1847.5137158	total: 17.6s	remaining: 2m 30s
40:	learn: 1764.1410750	total: 34.7s	remaining: 2m 14s
60:	learn: 1720.7518886	total: 51.5s	remaining: 1m 57s
80:	learn: 1694.4116347	total: 1m 9s	remaining: 1m 42s
100:	learn: 1675.7378133	total: 1m 26s	remaining: 1m 25s
120:	learn: 1661.6226607	total: 1m 44s	remaining: 1m 8s
140:	learn: 1646.9514413	total: 2m 1s	remaining: 51s
160:	learn: 1634.1239780	total: 2m 20s	remaining: 33.9s
180:	learn: 1624.2650841	total: 2m 37s	remaining: 16.5s
199:	learn: 1612.8882087	total: 2m 54s	remaining: 0us
1658.5511616619076


### Вывод
Были обучены различные модели, как и стандартные(линейная регрессия, решающее дерево, случайный лес), так и модели с градиентным бустингом(на основе библиотек Catboost и lightgbm. По итогу были получены значения RMSE для моделей с различными параметрами, и можно выбрать модели с наилучшими значениями.

## Анализ моделей

#### Линейная регрессия

In [25]:
model = LinearRegression(normalize = False)
result = %timeit -n1 -r1 -o model.fit(features_train, target_train)
result_1 = %timeit -n1 -r1 -o model.predict(features_test)
rmse_1 = mean_squared_error(target_test, model.predict(features_test))**0.5
print('RMSE модели', rmse_1)

24.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
295 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
RMSE модели 2872.1980533286855


#### Случайный лес

In [26]:
model = RandomForestRegressor(random_state=12345, n_estimators=10) 
result_2 = %timeit -n1 -r1 -o model.fit(features_train, target_train) 
result_3 = %timeit -n1 -r1 -o model.predict(features_test)
rmse_2 = mean_squared_error(target_test, model.predict(features_test))**0.5
print('RMSE модели', rmse_2)

1min 6s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
652 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
RMSE модели 1720.7039296795933


#### Решающее дерево

In [27]:
model = DecisionTreeRegressor(random_state=12345, max_depth = 10)
result_4 = %timeit -n1 -r1 -o model.fit(features_train, target_train) 
result_5 = %timeit -n1 -r1 -o model.predict(features_test)
rmse_3 = mean_squared_error(target_test, model.predict(features_test))**0.5
print('RMSE модели', rmse_3)

5.58 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
152 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
RMSE модели 2045.7635187134551


#### Модель с градиентным бустингом с использованием библиотеки lightgbm

In [28]:
model = LGBMModel(objective="regression", n_estimators=500, learning_rate = 0.4)
result_6 = %timeit -n1 -r1 -o model.fit(features_train, target_train, eval_set=[(features_test, target_test)], eval_metric = 'rmse', verbose = -1) 
result_7 = %timeit -n1 -r1 -o model.predict(features_test)
rmse_4 = mean_squared_error(target_test, model.predict(features_test))**0.5
print('RMSE модели', rmse_4)

35.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
3.69 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
RMSE модели 1612.6193692563127


#### Модель с градиентным бустингом с использованием библиотеки Catboost

In [29]:
model = CatBoostRegressor(loss_function="RMSE", iterations=200, learning_rate = 0.5) 
result_8 = %timeit -n1 -r1 -o model.fit(features_train_x, target_train_x, cat_features=cat_features, verbose=0)
result_9 = %timeit -n1 -r1 -o model.predict(features_test_x)
rmse_5 = mean_squared_error(target_test, model.predict(features_test_x))**0.5
print('RMSE модели', rmse_5)

2min 53s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
506 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
RMSE модели 1675.3936031913659


In [30]:
table = pd.DataFrame({'Модель': ['Линейная регрессия', 'Случайный лес', 'Решающее дерево', 'lightgbm', 'Catboost'],
                      'RMSE': [rmse_1, rmse_2, rmse_3, rmse_4, rmse_5], 
                      'Время обучения, c': [result.average, result_2.average, result_4.average, result_6.average, result_8.average], 
                      'Время предсказания, c': [result_1.average, result_3.average, result_5.average, result_7.average, result_9.average]})
table.set_index('Модель')

,RMSE,"Время обучения, c","Время предсказания, c"
Модель,,,
Линейная регрессия,2872.198053,24.170316,0.294612
Случайный лес,1720.703930,66.446554,0.651846
Решающее дерево,2045.763519,5.578280,0.152069
lightgbm,1612.619369,35.652822,3.687686
Catboost,1675.393603,173.594739,0.505603


### Вывод
По итогу можно сделать вывод о том, что модели с использованием градиентного бустинга показали более лучшее значение метрики RMSE, но при этом в среднем выполнение данных моделей занимает большее время, чем выполнение обычных моделей. Предпочтительней всего была бы модель с использованием библиотеки lightgbm